<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ViTMSN_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets transformers
!pip install wandb -qU
!pip install weightwatcher
!pip install accelerate -U

In [2]:
import wandb
wandb.login(relogin=True)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
%env WANDB_PROJECT=Funny_Not_Funny

env: WANDB_PROJECT=Funny_Not_Funny


In [4]:
wandb.init(
    # set the wandb project where this run will be logged
    project='Funny_Not_Funny',
    name='ViT_msn_Large'
)

wandb: Currently logged in as: vedaantj. Use `wandb login --relogin` to force relogin


In [5]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [6]:
%cd content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [7]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="Dataset/")

Resolving data files:   0%|          | 0/1615 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/539 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/539 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import weightwatcher as ww

In [9]:
from transformers import AutoImageProcessor
model_name_or_path = 'facebook/vit-msn-large'
image_processor  = AutoImageProcessor.from_pretrained(model_name_or_path)
image_processor

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [10]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = image_processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [11]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-11-223a1e4a98a9>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [12]:
from transformers import AutoImageProcessor, ViTMSNForImageClassification

labels = prepared_ds['train'].features['label'].names
model = ViTMSNForImageClassification.from_pretrained(model_name_or_path, num_labels=len(labels), id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}, ignore_mismatched_sizes=True)
NUM_CLASSES = 2
BATCH_SIZE = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Some weights of ViTMSNForImageClassification were not initialized from the model checkpoint at facebook/vit-msn-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTMSNForImageClassification(
  (vit): ViTMSNModel(
    (embeddings): ViTMSNEmbeddings(
      (patch_embeddings): ViTMSNPatchEmbeddings(
        (projection): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ViTMSNEncoder(
      (layer): ModuleList(
        (0-23): 24 x ViTMSNLayer(
          (attention): ViTMSNAttention(
            (attention): ViTMSNSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMSNSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): ViTMSNIntermediate(
  

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-msn",
  per_device_train_batch_size=BATCH_SIZE,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=10,
  learning_rate=1e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.1
)

In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=image_processor,
)

In [15]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
200,0.750300,0.846043,0.539889
400,0.644400,0.942776,0.525046
600,0.613100,0.668658,0.608534
800,0.605100,0.653903,0.653061


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1648082213GF
  train_loss               =       1.6104
  train_runtime            =   0:15:06.25
  train_samples_per_second =        7.128
  train_steps_per_second   =        0.892


In [16]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze()
summary = watcher.get_summary(details)
summary

{'log_norm': 2.676,
 'alpha': 6.281572311378339,
 'alpha_weighted': 4.71079153438089,
 'log_alpha_norm': 5.325686179734368,
 'log_spectral_norm': 0.9873,
 'stable_rank': 100.2}

In [17]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.6531
  eval_loss               =     0.6539
  eval_runtime            = 0:00:29.20
  eval_samples_per_second =     18.457
  eval_steps_per_second   =      2.329


In [18]:
metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)
wandb.finish()

***** eval metrics *****
  test_accuracy           =     0.5788
  test_loss               =     0.7043
  test_runtime            = 0:02:21.24
  test_samples_per_second =      3.816
  test_steps_per_second   =      0.481


eval/accuracy,▂▁▆██
eval/loss,▆█▁▁▁
eval/runtime,█▁██▆
eval/samples_per_second,▁█▁▁▂
eval/steps_per_second,▁█▁▁▃
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [19]:
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape

torch.Size([539, 2])


(tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
         0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
         1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
         1, 0, 0, 1, 0, 1, 1

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [21]:
print(accuracy_score(preds, metrics.label_ids), f1_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Test Funny', 'Test Not Funny'], index=['Pred Funny', 'Pred Not Funny'])

0.5194805194805194 0.3934426229508197


,Test Funny,Test Not Funny
Pred Funny,196,230
Pred Not Funny,29,84
